Ever wanted a recursive agent? Now you can have one! 🤯

In [1]:
# This page will use the following imports:

from __future__ import annotations

from lasagna import Model, EventCallback, AgentRun, Message
from lasagna import (
    override_system_prompt,
    extraction,
    chained_runs,
    to_str,
    make_model_binder,
    noop_callback,
)
from lasagna import known_models

from pydantic import BaseModel, Field

import io
import os
import sys
import copy
import asyncio
from contextlib import redirect_stdout

from dotenv import load_dotenv

We need to set up our "binder" (see the [quickstart guide](../quickstart.ipynb) for what this is).

In [2]:
load_dotenv()

if os.environ.get('OLLAMA'):
    model_name = os.environ['OLLAMA']
    print('Using Ollama model:', model_name)
    BINDER = make_model_binder('ollama', model_name)

elif os.environ.get('ANTHROPIC_API_KEY'):
    print('Using Anthropic')
    BINDER = known_models.anthropic_claude_haiku_4_5_binder

elif os.environ.get('OPENAI_API_KEY'):
    print('Using OpenAI')
    BINDER = known_models.openai_gpt_5_mini_binder

else:
    assert False, "Neither OLLAMA nor ANTHROPIC_API_KEY nor OPENAI_API_KEY is set! We need at least one to do this demo."

Using Anthropic


Ensure the recursion doesn't go forever!

In [3]:
MAX_DEPTH = int(os.environ.get('MAX_DEPTH', '3'))

## Example: Recursive Plot Outline Builder

Let's write a recursive agent that builds a plot outline. It will recursively add detail to the plot outline until it is sufficiently detailed (or until the `MAX_DEPTH` is reached).

### System Prompt

In [4]:
SYSTEM_PROMPT = """
You expand the plot outline to add detail, nuance, and time-tested
story-telling techniques.

You will be given a plot outline, and you will be instructed to expand
**one** part of it. Your output should be the expansion of that **single**
part.
""".strip()

### Data Models

The following data model will be used to _extract_ (i.e. generate structured output) plot revisions at each step in the recursive process:

In [5]:
class PlotRevision(BaseModel):
    thoughts: str = Field(description="your free-form thoughts about the entire plot and about the part you are asked to expand, and then brainstorm ideas for how it can be modified or expanded")
    revised_plot_point: str = Field(description="output a revised plot point that will **replace** the plot point you were instructed to expand; you may, if you choose, output the **same** plot point verbatim if it does not need modification")
    has_sufficient_detail: bool = Field(description="`true` if this plot point has sufficient detail to easily write the script from it; else `false` if this plot point needs to have further expounding; if `false`, then fill in `sub_plot_points` with the sub-plot points to add detail")
    sub_plot_points: list[str] = Field(description="output a chronological list of sub-plot points; these are details about the `revised_plot_point` to add detail, nuance, and time-tested story-telling techniques; output an empty list if `has_sufficient_detail` is `true`")

The following data model will be used to hold the _full_ plot (capturing everything generated up to this point). Note: It does not need prompts since it will _not_ be used for extraction by AI models.

In [6]:
class FullPlotPoint(BaseModel):
    plot_point: str
    sub_plot_points: list[FullPlotPoint]

### The Recursive Agent

Now, the recursive agent! See comments below for details, but the high level is that it:

1. generates a prompt to extract a `PlotRevision`,
2. patches the current `FullPlotPoint` using that `PlotRevision`, and
3. for each generated subplot, recurses to continue filling in lower-level detail.

In [7]:
@BINDER
async def plot_builder(
    model: Model,
    event_callback: EventCallback,
    prev_runs: list[AgentRun],
) -> AgentRun:
    # Capture all the generated output:
    output_runs: list[AgentRun] = []

    # Build the extraction input:
    assert len(prev_runs) == 2
    plot_agentrun = prev_runs[0]
    assert plot_agentrun['type'] == 'extraction'
    full_plot = copy.deepcopy(plot_agentrun['result'])   # COPYING IS IMPORTANT HERE! WE MUST BE A PURE FUNCTION!
    assert isinstance(full_plot, FullPlotPoint)
    revise_indices_agentrun = prev_runs[1]
    assert revise_indices_agentrun['type'] == 'extraction'
    revise_indices = revise_indices_agentrun['result']
    assert isinstance(revise_indices, list)
    specific_point = full_plot
    for index in revise_indices:
        assert isinstance(index, int)
        specific_point = specific_point.sub_plot_points[index]
    messages_orig: list[Message] = [
        {
            'role': 'human',
            'text': f'Given this plot outline: {full_plot}',
        },
        {
            'role': 'human',
            'text': f'Revise this plot point: {specific_point.plot_point}',
        },
    ]
    messages_new_system_prompt = override_system_prompt(messages_orig, SYSTEM_PROMPT)

    # Extract (generate with structured output) the plot revision at this level:
    extraction_message, extraction_result = await model.extract(
        event_callback,
        messages = messages_new_system_prompt,
        extraction_type = PlotRevision,
    )
    extraction_run = extraction('extraction_run', [extraction_message], extraction_result)
    output_runs.append(extraction_run)

    # Patch the plot:
    specific_point.plot_point = extraction_result.revised_plot_point
    specific_point.sub_plot_points = [
        FullPlotPoint(
            plot_point = point,
            sub_plot_points = [],
        )
        for point in extraction_result.sub_plot_points
    ]

    # Can we afford to go deeper?
    can_go_deeper = len(revise_indices) + 1 < MAX_DEPTH

    # Recursively expand all the sub-plot points:
    if can_go_deeper and not extraction_result.has_sufficient_detail:
        subplot_runs: list[AgentRun] = []
        for i, _ in enumerate(extraction_result.sub_plot_points):
            subplot_input: list[AgentRun] = [
                extraction('plot_agentrun', [], full_plot),
                extraction('revise_indices_agentrun', [], [*revise_indices, i]),
            ]
            subplot_run = await plot_builder(event_callback, subplot_input)
            subplot_runs.append(subplot_run)
        subplot_chain = chained_runs('subplot_chain', subplot_runs)
        output_runs.append(subplot_chain)

    # Return everything that we generated:
    return chained_runs('plot_builder', output_runs)

### Utility Functions

Here is a utility function to print the resulting plot outline using the final `AgentRun` returned by the agent above.

In [8]:
def print_plot(run: AgentRun, depth: int = 0) -> int:
    """
    Prints the full plot outline, and returns the number of _extractions_ that were generated along the way. 
    """
    assert run['agent'] == 'plot_builder'
    assert run['type'] == 'chain'
    extraction_run = run['runs'][0]
    assert extraction_run['agent'] == 'extraction_run'
    assert extraction_run['type'] == 'extraction'
    plot_revision = extraction_run['result']
    assert isinstance(plot_revision, PlotRevision)
    print(f"{' ' * 2*depth}- {plot_revision.revised_plot_point}")
    sum_ = 1  # count _this_ run
    if len(run['runs']) > 1:
        subplot_chain = run['runs'][1]
        assert subplot_chain['agent'] == 'subplot_chain'
        assert subplot_chain['type'] == 'chain'
        for subplot_run in subplot_chain['runs']:
            sum_ += print_plot(subplot_run, depth = depth + 1)
    else:
        for leaf_plot_point in plot_revision.sub_plot_points:
            print(f"{' ' * 2*(depth+1)}- {leaf_plot_point}")
    return sum_

### Driver

Here is a driver function to make use of the `plot_builder`.

It:

1. kicks off the first call to the `plot_builder`,
2. writes a file (for inspection outside this notebook) of the full `AgentRun` that is returned by the agent,
3. writes a file of the full plot outline generated by the agent, and
4. prints the full plot outline here in the notebook as well.

In [9]:
async def run_plot_builder(prompt: str) -> None:
    plot_agentrun = extraction('plot_agentrun', [], FullPlotPoint(
        plot_point = prompt,
        sub_plot_points = [],
    ))
    revise_indices_agentrun = extraction('revise_indices_agentrun', [], [])

    run = await plot_builder(noop_callback, [plot_agentrun, revise_indices_agentrun])

    with open('plot_builder_output.json', 'wt') as f:
        f.write(to_str(run))

    with redirect_stdout(io.StringIO()) as f:
        total_extractions = print_plot(run)
    plot_outline = f.getvalue()

    print(plot_outline)
    print(f"Did a total of {total_extractions} extractions.")

    with open('plot_builder_output.txt', 'wt') as f:
        f.write(plot_outline)

### Example Execution

In [10]:
PROMPT = """
A short film about a motocross race.
""".strip()

await run_plot_builder(PROMPT)  # type: ignore[top-level-await]

- A determined rider fights to redeem themselves at a prestigious motocross championship after a career-defining setback, navigating personal demons, fierce competition, and their fraught relationship with a rival they once mentored.
  - At the regional qualifier—billed as the final pathway to the nationals—the aging protagonist makes an uncharacteristic tactical error during a crucial heat, losing control on a line they've executed flawlessly for decades. They crash hard in front of the crowds and sponsors, then remount to finish out of the points entirely. Shaken and visibly injured, they face their lead sponsor in the pits afterward, who delivers an ultimatum: place in the top three at the nationals in three weeks, or the sponsorship deal—their primary source of income—is terminated.
    - In the immediate aftermath of the crash, the protagonist clings to a fortress of rationalizations, fixating on external culprits—a setup issue they swear was present all week, a slick patch of tra

## Other References